In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
import os
from itertools import product
from functions import (load_parameters, load_generation_data, load_price_data, generate_randomized_generation,
generate_rt_scenarios, plot_generation_data, plot_randomized_generation, plot_scenarios_for_generator, plot_rt_scenarios)

generation_data, I, T = load_generation_data(date_filter="2022-07-18")
S, R, P_RT, K, K0, M1, M2 = load_parameters(I, T, generation_data)
P_DA, P_PN = load_price_data()

✅ 총 5개 파일을 불러왔습니다: 1201.csv, 137.csv, 401.csv, 524.csv, 89.csv
📊 데이터 Shape: I=5, T=24, S=10
✅ 시뮬레이션 초기화 완료: S=10, Randomness='high', M1=758.40, M2=1855.82


/Users/jangseohyun/Documents/workspace/symply/DER/functions.py:95: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  nyc_rt["Hour"] = nyc_rt["Time Stamp"].dt.floor("H")


### 모델 DER only

In [2]:
only = gp.Model("only")
# only.setParam("MIPGap", 1e-7)

x = only.addVars(I, T, vtype=GRB.CONTINUOUS, lb=0, name="x")
y_plus = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus")
y_minus = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus")

M = max(R[i, t, s] for i in range(I) for t in range(T) for s in range(S))
z = only.addVars(I, T, S, vtype=GRB.BINARY, name="z")

only.update()

obj = gp.quicksum(
    P_DA[t] * x[i, t] for i in range(I) for t in range(T)
) + gp.quicksum(
    1/S * (P_RT[t, s] * y_plus[i, t, s] - P_PN[t] * y_minus[i, t, s])
    for i in range(I)
    for t in range(T)
    for s in range(S)
)

only.setObjective(obj, GRB.MAXIMIZE)

Set parameter Username
Set parameter LicenseID to value 2611964
Academic license - for non-commercial use only - expires 2026-01-20


In [3]:
for i in range(I):
    for t in range(T):
        for s in range(S):
            only.addConstr(R[i, t, s] - x[i, t] == y_plus[i, t, s] - y_minus[i, t, s])

for i in range(I):
    for t in range(T):
        for s in range(S):
            only.addConstr(y_plus[i, t, s] <= R[i, t, s])

# for i in range(I):
#     for t in range(T):
#         for s in range(S):
#             only.addConstr(y_plus[i, t, s] <= M * z[i, t, s])
#             only.addConstr(y_minus[i, t, s] <= M * (1 - z[i, t, s]))

only.optimize()

if only.status == GRB.OPTIMAL:
    print("Optimal solution found!")
    print(f"Objective value: {only.objVal}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.3.0 24D81)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2400 rows, 3720 columns and 4800 nonzeros
Model fingerprint: 0x70432a39
Variable types: 2520 continuous, 1200 integer (1200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-01, 8e+02]
Found heuristic solution: objective 1240294.1382
Presolve removed 2400 rows and 3720 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 3: 1.53e+06 1.53e+06 1.24029e+06 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.530004566343e+06, best bound 1.530004566343e+06, gap 0.0000%
Optimal solution found!
Objective value: 1530004.5663426

### 결과 분석

#### 수익 분석

In [4]:
# Day-ahead 수익 비교
total_da_profit_obj = 0
for i in range(I):
    for t in range(T):
        total_da_profit_obj += P_DA[t] * x[i,t].x

# Real-time 수익 비교
total_rt_profit_obj = 0
for i in range(I):
    for t in range(T):
        for s in range(S):
            rt_profit_obj = P_RT[t, s] * y_plus[i, t, s].x
            total_rt_profit_obj += 1/S * rt_profit_obj

# 패널티 비용 비교
total_penalty_cost_obj = 0
for i in range(I):
    for t in range(T):
        for s in range(S):
            penalty_cost_obj = P_PN[t] * y_minus[i, t, s].x
            total_penalty_cost_obj += 1/S * penalty_cost_obj

# 총 시스템 이익 (목적 함수 기반)
total_system_profit_obj = total_da_profit_obj + total_rt_profit_obj - total_penalty_cost_obj

# 결과 출력
print(f"DA: {total_da_profit_obj:.2f}")
print(f"RT: {total_rt_profit_obj:.2f}")
print(f"Penalty: {total_penalty_cost_obj:.2f}")
print(f"목적 함수 기반 총 이익 (_obj): {total_system_profit_obj:.2f}")

DA: 342680.80
RT: 1226388.80
Penalty: 39065.04
목적 함수 기반 총 이익 (_obj): 1530004.57


#### 하루 개인별 커밋량 합 분석

In [5]:
sum_x = 0
for t in range(T):
    time_sum = sum(x[i,t].x for i in range(I))
    print(f"시간 {t}: {time_sum}")
    sum_x += time_sum
print(f"총 하루 commitment: {sum_x:.2f}")


시간 0: 0.0
시간 1: 0.0
시간 2: 0.0
시간 3: 0.0
시간 4: 0.15229737203378113
시간 5: 0.0
시간 6: 1.3306829300336802
시간 7: 5.479519341905354
시간 8: 18.59900400654967
시간 9: 126.15673563965795
시간 10: 347.51976572781876
시간 11: 0.0
시간 12: 0.0
시간 13: 0.0
시간 14: 910.4313351392219
시간 15: 0.0
시간 16: 439.4541599522865
시간 17: 0.0
시간 18: 0.0
시간 19: 354.3881283326069
시간 20: 85.29416868547565
시간 21: 11.95395868138966
시간 22: 0.0
시간 23: 0.0
총 하루 commitment: 2300.76


#### 개인 수익 분석

In [6]:
# 모든 der의 profit 합계 계산
total_der_profit = 0
der_profit = {}
for i in range(I):
    # 각 der i의 profit 합계 계산
    der_profit[i] = sum(P_DA[t] * x[i,t].x + sum(1/S * (P_RT[t,s] * y_plus[i,t,s].x - P_PN[t] * y_minus[i,t,s].x) for s in range(S)) for t in range(T))
    total_der_profit += der_profit[i]

print("\nder_profit:")
for i in range(I):
    print(f"[{i}] {der_profit[i]:.2f}")

print(f"\n모든 der의 profit 합계: {total_der_profit:.2f}")

der_hourly_profit = np.zeros((I, T, 4))

for i in range(I):
    for t in range(T):
        da_profit = P_DA[t] * x[i, t].x
        rt_profit = sum(1 / S * (P_RT[t, s] * y_plus[i, t, s].x) for s in range(S))
        pen_cost = sum(1 / S * (P_PN[t] * y_minus[i, t, s].x) for s in range(S))
        hourly_total = da_profit + rt_profit - pen_cost

        der_hourly_profit[i, t, 0] = da_profit  # DA profit
        der_hourly_profit[i, t, 1] = rt_profit  # RT profit
        der_hourly_profit[i, t, 2] = pen_cost   # Penalty cost
        der_hourly_profit[i, t, 3] = hourly_total



der_profit:
[0] 203054.79
[1] 333630.26
[2] 373699.78
[3] 451200.67
[4] 168419.07

모든 der의 profit 합계: 1530004.57


### 결과 저장

In [7]:
# R_sum = {(i, t): sum(R[i, t, s] for s in range(S)) / S for i in range(I) for t in range(T)}
# R_df = pd.DataFrame({i: {t: R_sum[i, t] for t in range(T)} for i in range(I)})
# R_df.to_csv('result_R.csv', index=False)

# pd.DataFrame(list(P_DA.items()), columns=['time', 'price']).to_csv('result_P_DA.csv', index=False)

# total_x_only = {t: sum(x[i,t].x for i in range(I)) for t in range(T)}
# total_x_only_df = pd.DataFrame({'total_x_only': total_x_only})
# total_x_only_df.to_csv('result/result_base_totalX.csv', index=False)

only_obj = pd.DataFrame({'only_obj': [only.objVal]})
only_obj.to_csv('result/result_only_obj.csv', index=False)

only_profit_value = pd.DataFrame({'only_profit_value': der_profit})
only_profit_value.to_csv('result/result_only_profit.csv', index=False)

data_list = []

for i in range(I):
    for t in range(T):
        data_list.append([i, t, 
                          der_hourly_profit[i, t, 0],  # DA profit
                          der_hourly_profit[i, t, 1],  # RT profit
                          der_hourly_profit[i, t, 2],  # Penalty cost
                          der_hourly_profit[i, t, 3]]) # Hourly total profit

# Pandas DataFrame 생성
df = pd.DataFrame(data_list, columns=['DER', 'Hour', 'hourly_da', 'hourly_rt', 'hourly_pen', 'hourly_total'])

# CSV 파일로 저장
df.to_csv('result/result_only_hourly_profit.csv', index=False)
